In [27]:
import uproot
import numpy as np
import awkward
import concurrent.futures
import math
import collections

In [61]:
'''LOAD THE FILE AND ITS CONTENTS'''

filename = 'output_files_total/0.01_trigger_BDT_RoC_hough.root'

branches = ['ax','ay','az','bx','by','bz','nlines','hitX','hitY','hitZ','pdgID','pdg_assoc','distFromETraj','distFromPTraj', 'x_assoc', 'y_assoc','z_assoc']

t = uproot.open(filename)['Events']
table = t.arrays(expressions=branches)

tree = {}
for branch in branches:
    tree[branch] = table[branch]

In [62]:
'''COUNT THE NUMBER OF TRACKS PER EVENT'''
trackCounter = np.array(tree['nlines']) 

'''LOAD LAYER INFORMATION AND CONSTRUCT VARIABLE ARRAYS'''
layer_dz = np.array([7.850, 13.300, 26.400, 33.500, 47.950, 56.550, 72.250, 81.350, 97.050, 106.150,
            121.850, 130.950, 146.650, 155.750, 171.450, 180.550, 196.250, 205.350, 221.050,
            230.150, 245.850, 254.950, 270.650, 279.750, 298.950, 311.550, 330.750, 343.350,
            362.550, 375.150, 394.350, 406.950, 426.150, 438.750])

layer_z = 240.5 + layer_dz

fracLayers = []
inARow = []
numGaps = []


In [63]:
# 1. Fraction Layers with Multiple Hits -> Eliminate Tracks with Fraction > 0
for event in range(len(tree['z_assoc'])):

    fractions = []
    for track in range(len(tree['z_assoc'][event])):
        layers = sorted(tree['z_assoc'][event][track])
        start = 0
        # find the index of which ECal layer to start from
        for i in range(len(layer_z)):
            if abs(layer_z[i] - layers[0]) < 0.01:
                start = i
                break
        end = 0
        # find the index of which ECal layer to end from
        for j in range(len(layer_z)):
            if abs(layer_z[j] - layers[-1]) < 0.01:
                end = j
                break
        # find the number of layers from start to end
        interval = end - start + 1

        # list of layer z positions that have more than 1 hit in the track
        repeatedZs = [z for z,count in collections.Counter(tree['z_assoc'][event][track]).items() if count > 1]
        
        # fraction of layers with multiple hits
        fraction = len(repeatedZs)/interval
        fractions.append(fraction)

    fracLayers.append(fractions)  

In [64]:
# 2. Number of Layers in a Row with a Single Hit -> Eliminate Tracks with Layers < 3
for event in range(len(tree['z_assoc'])):

    maximums = []
    for track in range(len(tree['z_assoc'][event])):
        # list of layer z positions that have more than 1 hit in the track
        repeatedZs = [z for z,count in collections.Counter(tree['z_assoc'][event][track]).items() if count > 1]
        counter = 0
        maximum = 0
        hitZ = sorted((tree['z_assoc'][event][track]))
        for hit in hitZ: 
            if not(hit in repeatedZs):
                counter += 1
                if (counter > maximum):
                    maximum = counter 
            elif (hit in repeatedZs and counter > maximum):
                maximum = counter
                counter = 0
            else:
                counter = 0
        
        maximums.append(maximum)
    
    inARow.append(maximums)

In [65]:
# 3. Number of Layer Gaps -> Eliminate Tracks with Number of Layer Gaps > 1
for event in range(len(tree['z_assoc'])):
    
    gaps = []
    for track in range(len(tree['z_assoc'][event])):
        hitZ = sorted((tree['z_assoc'][event][track]))
        trackLayers = sorted([*set(hitZ)])
        #print("trackLayers: {}".format(trackLayers))
        
        start = 0
        # find the index of which ECal layer to start from
        for i in range(len(layer_z)):
            if abs(layer_z[i] - trackLayers[0]) < 0.01:
                start = i
                break
        end = 0
        # find the index of which ECal layer to end from
        for j in range(len(layer_z)):
            if abs(layer_z[j] - trackLayers[-1]) < 0.01:
                end = j
                break
        # find the number of layers from start to end
        interval = end - start + 1
        gap = interval - len(trackLayers)
        gaps.append(gap)
    
    numGaps.append(gaps)

In [66]:
'''IMPLEMENT CUTS ON THE SIGNAL TRACKS'''
for event in range(len(tree['nlines'])):
    for track in range(len(fracLayers[event])):
        if (fracLayers[event][track] > 0.4 or inARow[event][track] == 0 or numGaps[event][track] > 4):
            trackCounter[event] -= 1


In [67]:
'''CHECK HOW MANY SIGNAL EVENTS HAVE TRACKS'''
counter = 0
for element in trackCounter:
    if element != 0:
        counter += 1
print("Number of signal events with a track found: {}".format(counter))

Number of signal events with a track found: 151920


In [60]:
print(sum(trackCounter))

133513
